In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [3]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [4]:
train_X

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
0,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,0,0,1
1,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,0,1,0
2,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,0,1,0
3,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,0,0,1
4,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1248,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.0150,0,1,0
1249,0.395,0.310,0.085,0.3170,0.1530,0.0505,0.0935,0,1,0
1250,0.525,0.410,0.115,0.7745,0.4160,0.1630,0.1800,1,0,0
1251,0.445,0.335,0.110,0.4355,0.2025,0.1095,0.1195,1,0,0


In [5]:
# 전복 전체 무게 대비 불필요한 물질들의 값 ( 이물질 / 핏물 등)
foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])

In [6]:
foreign_body

0       0.1205
1       0.0185
2       0.1220
3       0.0590
4       0.0080
         ...  
1248    0.0000
1249    0.0200
1250    0.0155
1251    0.0040
1252    0.1965
Length: 1253, dtype: float64

In [7]:
foreign_body[47]

-0.06749999999999998

In [8]:
train_X['foreign body'] = foreign_body

In [9]:
train_X.loc[train_X['foreign body'] < 0 , 'foreign body'] = 0

In [10]:
# 테스트셋에도 적용
foreign_body = test['Whole Weight'] - (test['Shucked Weight'] + test['Viscra Weight'] + test['Shell Weight'])
test['foreign body'] = foreign_body
test.loc[test['foreign body'] < 0 , 'foreign body'] = 0

In [11]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [94]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=11, activation='elu'))
model.add(Dense(32,  activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))

model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

In [95]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = MODEL_DIR + "{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

In [96]:
# 모델 학습
model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

Epoch 1/1000
28/28 [==============================] - 1s 9ms/step - loss: 6.0501 - mae: 6.0501 - val_loss: 2.1513 - val_mae: 2.1513 - lr: 0.0010
Epoch 2/1000
28/28 [==============================] - 0s 4ms/step - loss: 2.2264 - mae: 2.2264 - val_loss: 1.9263 - val_mae: 1.9263 - lr: 0.0010
Epoch 3/1000
28/28 [==============================] - 0s 3ms/step - loss: 2.1204 - mae: 2.1204 - val_loss: 1.9531 - val_mae: 1.9531 - lr: 0.0010
Epoch 4/1000
28/28 [==============================] - 0s 4ms/step - loss: 1.9960 - mae: 1.9960 - val_loss: 1.8990 - val_mae: 1.8990 - lr: 0.0010
Epoch 5/1000
28/28 [==============================] - 0s 4ms/step - loss: 2.0176 - mae: 2.0176 - val_loss: 1.8963 - val_mae: 1.8963 - lr: 0.0010
Epoch 6/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.9007 - mae: 1.9007 - val_loss: 1.9110 - val_mae: 1.9110 - lr: 0.0010
Epoch 7/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.9039 - mae: 1.9039 - val_loss: 1.9011 - val_mae: 1.901

28/28 [==============================] - 0s 3ms/step - loss: 1.4668 - mae: 1.4668 - val_loss: 1.5902 - val_mae: 1.5902 - lr: 0.0010
Epoch 114/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4367 - mae: 1.4367 - val_loss: 1.5621 - val_mae: 1.5621 - lr: 0.0010
Epoch 115/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4776 - mae: 1.4776 - val_loss: 1.5587 - val_mae: 1.5587 - lr: 0.0010
Epoch 116/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4369 - mae: 1.4369 - val_loss: 1.6057 - val_mae: 1.6057 - lr: 0.0010
Epoch 117/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4724 - mae: 1.4724 - val_loss: 1.5822 - val_mae: 1.5822 - lr: 0.0010
Epoch 118/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4472 - mae: 1.4472 - val_loss: 1.6545 - val_mae: 1.6545 - lr: 0.0010
Epoch 119/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4439 - mae: 1.4439 - val_loss: 1.5754 - val_mae: 1.5754

Epoch 169/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.3983 - mae: 1.3983 - val_loss: 1.5549 - val_mae: 1.5549 - lr: 0.0010
Epoch 170/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4325 - mae: 1.4325 - val_loss: 1.5566 - val_mae: 1.5566 - lr: 0.0010
Epoch 171/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4087 - mae: 1.4087 - val_loss: 1.5495 - val_mae: 1.5495 - lr: 0.0010
Epoch 172/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4031 - mae: 1.4031 - val_loss: 1.5540 - val_mae: 1.5540 - lr: 0.0010
Epoch 173/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4070 - mae: 1.4070 - val_loss: 1.5646 - val_mae: 1.5646 - lr: 0.0010
Epoch 174/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4233 - mae: 1.4233 - val_loss: 1.5309 - val_mae: 1.5309 - lr: 0.0010
Epoch 175/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4188 - mae: 1.4188 - val_loss: 1.5281 - 

Epoch 225/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.3853 - mae: 1.3853 - val_loss: 1.5370 - val_mae: 1.5370 - lr: 0.0010
Epoch 226/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.3996 - mae: 1.3996 - val_loss: 1.5534 - val_mae: 1.5534 - lr: 0.0010
Epoch 227/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.3944 - mae: 1.3944 - val_loss: 1.5614 - val_mae: 1.5614 - lr: 0.0010
Epoch 228/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.3819 - mae: 1.3819 - val_loss: 1.5674 - val_mae: 1.5674 - lr: 0.0010
Epoch 229/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.3748 - mae: 1.3748 - val_loss: 1.5461 - val_mae: 1.5461 - lr: 0.0010
Epoch 230/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4174 - mae: 1.4174 - val_loss: 1.5609 - val_mae: 1.5609 - lr: 0.0010
Epoch 231/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.3848 - mae: 1.3848 - val_loss: 1.5412 - 

In [87]:
Y_prediction = model.predict(test).flatten()

In [88]:
Y_prediction

array([10.098623 , 15.096386 ,  5.2617126, ...,  9.307539 ,  8.53819  ,
       11.744217 ], dtype=float32)

In [89]:
submission = pd.read_csv('./data/sample_submission.csv')

submission['Target'] = np.round(Y_prediction)

submission

,id,Target
0,1,10.0
1,2,15.0
2,3,5.0
3,4,11.0
4,5,9.0
...,...,...
2919,2920,4.0
2920,2921,7.0
2921,2922,9.0
2922,2923,8.0


In [90]:
submission.to_csv('./result/elu.csv',index=False)